# All geometries
> Comparison to decision trees/random forests in Hyperboloid, Klein, and Poincare ball models

In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from hyperdt.toy_data import wrapped_normal_mixture
from hyperdt.tree import HyperbolicDecisionTreeClassifier
from hyperdt.forest import HyperbolicRandomForestClassifier
from hyperdt.conversions import convert

import sys
from tqdm.notebook import tqdm
import time

sys.path.append("..")
from HoroRF.datasets.gaussian import get_training_data

from geomstats.geometry.hyperboloid import Hyperboloid
from geomstats.learning.pca import TangentPCA
from geomstats.learning.frechet_mean import FrechetMean

In [83]:
# %%prun

results = pd.DataFrame(columns=["seed", "n_dim", "model", "geometry", "f1_score", "time"])

seeds = list(range(10))
dims = [2, 4, 8, 16]
clfs = [
    HyperbolicDecisionTreeClassifier,
    HyperbolicRandomForestClassifier,
    DecisionTreeClassifier,
    RandomForestClassifier,
]
# geometries = ["euclidean", "tpca", "poincare", "hyperboloid", "klein"]
geometries = ["euclidean", "poincare", "hyperboloid", "klein"]
my_tqdm = tqdm(total=len(seeds) * len(dims) * len(clfs) * len(geometries) * 5)


for n_dim in dims:
    manifold = Hyperbolic(dim=n_dim, default_coords_type="extrinsic")
    metric = manifold.metric
    origin = np.array([1.0] + [0.0] * n_dim)

    for seed in seeds:
        my_tqdm.set_description(f"{n_dim}, {seed}")
        # X, y = get_training_data(class_label=n_dim, seed=seed, num_samples=int(800 / 0.8), convert_to_poincare=False)
        X, y = wrapped_normal_mixture(num_points=1000, n_dim=n_dim, num_classes=6, seed=seed, noise_std=2.0)
        # X, y = X.numpy(), y.numpy()

        # Both models like hyperboloids, so this is easy
        folds = KFold(n_splits=5, shuffle=True, random_state=seed)

        for train_index, test_index in folds.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            for geometry in geometries:
                if geometry in ["poincare", "klein", "hyperboloid"]:
                    X_train_g = convert(X_train, initial="hyperboloid", final=geometry)
                    X_test_g = convert(X_test, initial="hyperboloid", final=geometry)
                elif geometry == "tpca":
                    # Tangent PCA
                    mean = FrechetMean(metric)
                    mean.fit(X_train)
                    tpca = TangentPCA(metric, n_components=n_dim)
                    tpca.fit(X_train, base_point=mean.estimate_)
                    X_train_g = tpca.transform(X_train)
                    X_test_g = tpca.transform(X_test)
                elif geometry == "euclidean":
                    # Euclidean : log map to origin tangent space
                    X_train_g = manifold.metric.log(X_train, base_point=origin)
                    X_test_g = manifold.metric.log(X_test, base_point=origin)

                for clf_class in clfs:
                    try:
                        t1 = time.time()
                        if clf in [RandomForestClassifier, HyperbolicRandomForestClassifier]:
                            clf = clf_class(max_depth=3, n_estimators=12, random_state=seed, min_samples_leaf=1)
                        else:
                            clf = clf_class(max_depth=3, min_samples_leaf=1)
                        clf.fit(X_train_g, y_train)
                        y_pred = clf.predict(X_test_g)
                        t2 = time.time()
                        score = f1_score(y_test, y_pred, average="micro")
                        results.loc[len(results)] = [seed, n_dim, clf_class.__name__, geometry, score, t2 - t1]
                    except ValueError:
                        pass  # Should happen when the model doesn't support the geometry

                    my_tqdm.update(1)

  0%|          | 0/3200 [00:00<?, ?it/s]

In [92]:
# results.to_csv("../data/processed/all_geometries_6class.csv", index=False)
results = pd.read_csv("../data/processed/all_geometries.csv")

In [93]:
pd.pivot_table(results, index=["n_dim", "geometry"], columns=["model"], values=["f1_score"], aggfunc=np.mean) * 100

f1_score                                   \
model             DecisionTreeClassifier HyperbolicDecisionTreeClassifier   
n_dim geometry                                                              
2     euclidean                  91.8625                              NaN   
      hyperboloid                90.1000                          91.8750   
      klein                      91.8625                              NaN   
      poincare                   91.8500                              NaN   
4     euclidean                  99.0875                              NaN   
      hyperboloid                98.3500                          99.3000   
      klein                      99.2625                              NaN   
      poincare                   99.2875                              NaN   
8     euclidean                  99.9500                              NaN   
      hyperboloid                99.8875                          99.9625   
      klein                      99.9500                              NaN   
      poincare                   99.9625                              NaN   
16    euclidean                  99.9750                              NaN   
      hyperboloid               100.0000                         100.0000   
      klein                     100.0000                              NaN   
      poincare                   99.9875                              NaN   

                                                                           
model             HyperbolicRandomForestClassifier RandomForestClassifier  
n_dim geometry                                                             
2     euclidean                                NaN                92.2125  
      hyperboloid                          91.8000                89.5750  
      klein                                    NaN                92.1125  
      poincare                                 NaN                92.2500  
4     euclidean                                NaN                99.2875  
      hyperboloid                          99.4000                98.3625  
      klein                                    NaN                99.4750  
      poincare                                 NaN                99.4375  
8     euclidean                                NaN                99.9750  
      hyperboloid                          99.9625                99.9375  
      klein                                    NaN                99.9750  
      poincare                                 NaN                99.9875  
16    euclidean                                NaN               100.0000  
      hyperboloid                         100.0000               100.0000  
      klein                                    NaN               100.0000  
      poincare                                 NaN               100.0000

In [94]:
# Merge "model" and "geometry"

results_merged = results.copy()
results_merged["model"] = results_merged["model"] + "_" + results_merged["geometry"]
results_merged = results_merged.drop(columns=["geometry"])

pivot = pd.pivot_table(results_merged, index=["model"], columns=["n_dim"], values=["f1_score"], aggfunc=np.mean) * 100
pivot

f1_score                    \
n_dim                                              2        4        8    
model                                                                     
DecisionTreeClassifier_euclidean              91.8625  99.0875  99.9500   
DecisionTreeClassifier_hyperboloid            90.1000  98.3500  99.8875   
DecisionTreeClassifier_klein                  91.8625  99.2625  99.9500   
DecisionTreeClassifier_poincare               91.8500  99.2875  99.9625   
HyperbolicDecisionTreeClassifier_hyperboloid  91.8750  99.3000  99.9625   
HyperbolicRandomForestClassifier_hyperboloid  91.8000  99.4000  99.9625   
RandomForestClassifier_euclidean              92.2125  99.2875  99.9750   
RandomForestClassifier_hyperboloid            89.5750  98.3625  99.9375   
RandomForestClassifier_klein                  92.1125  99.4750  99.9750   
RandomForestClassifier_poincare               92.2500  99.4375  99.9875   

                                                        
n_dim                                               16  
model                                                   
DecisionTreeClassifier_euclidean               99.9750  
DecisionTreeClassifier_hyperboloid            100.0000  
DecisionTreeClassifier_klein                  100.0000  
DecisionTreeClassifier_poincare                99.9875  
HyperbolicDecisionTreeClassifier_hyperboloid  100.0000  
HyperbolicRandomForestClassifier_hyperboloid  100.0000  
RandomForestClassifier_euclidean              100.0000  
RandomForestClassifier_hyperboloid            100.0000  
RandomForestClassifier_klein                  100.0000  
RandomForestClassifier_poincare               100.0000

In [138]:
# We need to throw some of these out because the run failed

rg = results.groupby(["model", "seed", "geometry", "n_dim"]).count()["f1_score"]
rg[rg < 5]

model                             seed  geometry     n_dim
HyperbolicDecisionTreeClassifier  8     hyperboloid  16       1
HyperbolicRandomForestClassifier  8     hyperboloid  16       1
Name: f1_score, dtype: int64

In [144]:
print(len(results))
results2 = results[(results["seed"] != 8) | (results["n_dim"] != 16)]
# Only get rid of (seed=8, n_dim=16) because it's the only one that failed for all models
# This should get rid of
print(len(results2))

1987
1950


In [149]:
# A bunch of t-tests:
from scipy.stats import ttest_rel
from warnings import filterwarnings

# Get rid of the pandas UserWarning
filterwarnings("ignore")

# Clean up results: each (n_dim, model, trial) pair should have 5 entries

for model in results2["model"].unique():
    if "Hyperbolic" in model:
        continue
    for geom in results2["geometry"].unique():
        for n_dim in results2["n_dim"].unique():
            df_filtered = results2[
                (results["model"] == model) & (results["geometry"] == geom) & (results["n_dim"] == n_dim)
            ]
            # print(len(df_filtered))
            df_matched = results2[
                (results["model"] == f"Hyperbolic{model}")
                & (results["geometry"] == "hyperboloid")
                & (results["n_dim"] == n_dim)
            ]
            try:
                t, p = ttest_rel(df_filtered["f1_score"], df_matched["f1_score"])
            except ValueError as e:
                print(e)
            if p < 0.05 and df_filtered["f1_score"].mean() > df_matched["f1_score"].mean():
                print(f"{model} {geom} {n_dim}\t> Hyperbolic{model} with p={p:.4f}")
            elif p < 0.05 and df_filtered["f1_score"].mean() < df_matched["f1_score"].mean():
                print(f"{model} {geom} {n_dim}\t< Hyperbolic{model} with p={p:.4f}")
            else:
                continue

DecisionTreeClassifier hyperboloid 2	< HyperbolicDecisionTreeClassifier with p=0.0012
DecisionTreeClassifier hyperboloid 4	< HyperbolicDecisionTreeClassifier with p=0.0036
RandomForestClassifier poincare 2	> HyperbolicRandomForestClassifier with p=0.0137
RandomForestClassifier hyperboloid 2	< HyperbolicRandomForestClassifier with p=0.0032
RandomForestClassifier hyperboloid 4	< HyperbolicRandomForestClassifier with p=0.0010
RandomForestClassifier klein 2	> HyperbolicRandomForestClassifier with p=0.0434
RandomForestClassifier klein 4	> HyperbolicRandomForestClassifier with p=0.0128
